In [ ]:
import os
import pandas as pd
import shutil

csv_file = "dataset.csv"  
image_folder_initial = "imagenes/extracted_images"  
image_folder_resize = "imagenes/resize1024" 
output_csv = "dataset_updated.csv"  

df = pd.read_csv(csv_file)

def process_images(df, image_folder_initial, image_folder_resize, output_csv):
    updated_rows = [] 
    
    for index, row in df.iterrows():
        corrected_name = row['file_name'].replace("_resize1024", "")  
        folder, filename = corrected_name.split("/")  
        full_path_corrected = os.path.join(image_folder_initial, corrected_name)
        
        if os.path.exists(full_path_corrected):
            updated_rows.append(row)  
            continue
        
        base_name, rest_of_name = filename.split("_", 1)  
        partial_match = rest_of_name  
        
        folder_path_initial = os.path.join(image_folder_initial, folder)
        found = False
        for file in os.listdir(folder_path_initial):
            if partial_match in file: 
                old_path = os.path.join(folder_path_initial, file)
                new_path = os.path.join(folder_path_initial, filename)
                
                os.rename(old_path, new_path)
                print(f"Archivo renombrado: {file} -> {filename}")
                updated_rows.append(row)  
                found = True
                break
        
        if not found:
            folder_path_resize = os.path.join(image_folder_resize, folder)
            if os.path.exists(folder_path_resize):
                for file in os.listdir(folder_path_resize):
                    if partial_match in file:  
                        old_path = os.path.join(folder_path_resize, file)
                        new_path = os.path.join(folder_path_initial, filename)
                        
                        shutil.copy(old_path, new_path)
                        print(f"Archivo copiado y renombrado: {file} -> {filename}")
                        
                        updated_row = row.copy()
                        updated_row['file_name'] = corrected_name
                        updated_rows.append(updated_row)
                        found = True
                        break
        
        if not found:
            print(f"No se encontró coincidencia para: {corrected_name}")
            updated_rows.append(row)  
    
    updated_df = pd.DataFrame(updated_rows)
    
    updated_df['file_name'] = updated_df['file_name'].str.replace('_resize1024', '')

    updated_df.to_csv(output_csv, index=False)
    print(f"Nuevo CSV guardado en: {output_csv}")

process_images(df, image_folder_initial, image_folder_resize, output_csv)

Nuevo CSV guardado en: dataset_updated.csv


In [ ]:
import os
import pandas as pd

csv_file = "dataset_updated.csv" 
image_folder = "imagenes/extracted_images"  

df = pd.read_csv(csv_file)

def check_images_exist(df, image_folder):
    for index, row in df.iterrows():
        corrected_name = row['file_name']  
        folder, filename = corrected_name.split("/") 
        full_path_corrected = os.path.join(image_folder, corrected_name)
        
        if os.path.exists(full_path_corrected):
            continue
        else:
            print(f"Archivo NO encontrado: {corrected_name}")

check_images_exist(df, image_folder)

In [ ]:
import pandas as pd
from PIL import Image
import imagehash
import os

df = pd.read_csv('dataset_updated.csv')

def calculate_phash(image_path):
    try:
        image = Image.open(image_path)
        phash_value = imagehash.phash(image)
        return str(phash_value)
    except Exception as e:
        print(f"Error al procesar la imagen {image_path}: {e}")
        return None

base_path = "imagenes/extracted_images" 

for index, row in df.iterrows():
    if pd.isnull(row['phash']):
        image_path = os.path.join(base_path, row['file_name'])
        phash_value = calculate_phash(image_path)
        df.at[index, 'phash'] = phash_value

# Guardar el dataset actualizado
df.to_csv('tu_archivo_actualizado.csv', index=False)


Filas con 'subset' vacío: 0
Filas con 'genre' vacío: 0
Filas con 'artist' vacío: 0
Filas con 'artist' vacío: 0


In [ ]:
def emptyRows(df):
    empty_subset = df[df['subset'].isnull() | (df['subset'] == '')]
    empty_genre = df[df['genre'].isnull() | (df['genre'] == '')]
    empty_artist = df[df['artist'].isnull() | (df['artist'] == '')]
    empty_phash = df[df['phash'].isnull() | (df['phash'] == '')]

    print(f"Filas con 'subset' vacío: {len(empty_subset)}")
    print(f"Filas con 'genre' vacío: {len(empty_genre)}")
    print(f"Filas con 'artist' vacío: {len(empty_artist)}")
    print(f"Filas con 'artist' vacío: {len(empty_phash)}")

    if not empty_subset.empty:
        print("\nEjemplo de filas con 'subset' vacío:")
        display(empty_subset)

    if not empty_genre.empty:
        print("\nEjemplo de filas con 'genre' vacío:")
        display(empty_genre)

    if not empty_artist.empty:
        print("\nEjemplo de filas con 'artist' vacío:")
        display(empty_artist)
        
    if not empty_phash.empty:
        print("\nEjemplo de filas con 'phash' vacío:")
        display(empty_phash)

df = pd.read_csv('dataset.csv')

emptyRows(df)

Filas con 'subset' vacío: 0
Filas con 'genre' vacío: 0
Filas con 'artist' vacío: 0
Filas con 'artist' vacío: 2097

Ejemplo de filas con 'phash' vacío:


,file_name,genre,artist,painting_name,phash,description,subset
26,Realism/henry-herbert-la-thangue_stumping-the-...,Realism,henry herbert la thangue,stumping the cow,NaN,a painting of a man with a shovel and a cow,train
40,Baroque/mestre-ataíde_a-flagela-o-de-cristo_re...,Baroque,mestre ataíde,a flagela o de cristo,NaN,a painting of jesus with two men and a woman,train
49,Color_Field_Painting/gotthard-graubner_untitle...,Color Field Painting,gotthard graubner,untitled 1964 1,NaN,a painting with red; yellow; blue and white pa...,train
85,Expressionism/gulacsy-lajos_emily-1903_resize1...,Expressionism,gulacsy lajos,emily 1903,NaN,a painting of a woman with dark hair,train
106,Expressionism/salvador-dali_two-religious-figu...,Expressionism,salvador dali,two religious figures,NaN,a black and white drawing of two people,train
...,...,...,...,...,...,...,...
81307,Abstract_Expressionism/salvador-dali_cherub-su...,Abstract Expressionism,salvador dali,cherub super limen domus ezekiel 9 3 1967,NaN,a painting of a dog and a man in a red shirt,test
81340,Cubism/felix-del-marle_the-port-1914_resize102...,Cubism,felix del marle,the port 1914,NaN,a painting of a ship in the water with other o...,test
81354,Realism/martiros-saryan_from-the-artist-s-life...,Realism,martiros saryan,from the artist s life portrait of lusik sarya...,NaN,a painting of two women sitting at a table wit...,test
81366,Expressionism/miriam-schapiro_yard-sale-1993_r...,Expressionism,miriam schapiro,yard sale 1993,NaN,a painting of clothes hanging on a clothesline,test


In [3]:
import pandas as pd

df_correcto = pd.read_csv('.csv')

phash_map = dict(zip(df_correcto['file_name'], df_correcto['phash']))

def actualizar_phash(df, phash_map):
    df['file_name_base'] = df['file_name'].str.replace('_resize1024', '')
    df['phash'] = df['file_name_base'].map(phash_map)
    df.drop(columns=['file_name_base'], inplace=True)
    return df

# Leer los otros dos archivos CSV
df_archivo1 = pd.read_csv('dataset.csv')

# Actualizar los DataFrames con los valores de phash
df_archivo1 = actualizar_phash(df_archivo1, phash_map)

# Guardar los archivos actualizados
df_archivo1.to_csv('archivo1_actualizado.csv', index=False)

emptyRows(df_archivo1)

Filas con 'subset' vacío: 0
Filas con 'genre' vacío: 0
Filas con 'artist' vacío: 0
Filas con 'artist' vacío: 0


In [ ]:
csv_file = "archivo1_actualizado.csv"  
output_csv = "dataset768.csv" 

df = pd.read_csv(csv_file)
df['file_name'] = df['file_name'].str.replace('_resize1024', '_resize768')

df.to_csv(output_csv, index=False)



In [5]:
import os
import pandas as pd

csv_file = "dataset768.csv" 
image_folder = "imagenes/resize768"  

df = pd.read_csv(csv_file)

def check_images_exist(df, image_folder):
    for index, row in df.iterrows():
        corrected_name = row['file_name']  
        folder, filename = corrected_name.split("/") 
        full_path_corrected = os.path.join(image_folder, corrected_name)
        
        if os.path.exists(full_path_corrected):
            continue
        else:
            print(f"Archivo NO encontrado: {corrected_name}")

check_images_exist(df, image_folder)